In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split


In [2]:
# // Load data
perc_hold = 0.125
perc_test = 0.2
# the name of columns of the dataset
columns = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "capital_gain",
    "capital_loss",
    "work_hours",
    "native_country",
    "inc",
]

# here we shuffle train and test set
df_train = pd.read_csv("data/raw/adult.data", names=columns)
df_test = pd.read_csv("data/raw/adult.test", names=columns, skiprows=1)
df_train["TARGET"] = np.where(df_train["inc"] == " >50K", 1, 0)
df_test["TARGET"] = np.where(df_test["inc"] == " >50K.", 1, 0)
df_train.drop(columns=["inc", "education"], inplace=True) # here we drop additional columns that we do not need
df_test.drop(columns=["inc", "education"], inplace=True) # here we drop additional columns that we do not need
df = pd.concat([df_train, df_test], axis=0) #concat the two dataset
df.reset_index(inplace=True) #reset index
#here we define the final columns of the dataset
atts = [
    "age",
    "workclass",
    "fnlwgt",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "capital_gain",
    "capital_loss",
    "work_hours",
    "native_country",
]
# here we distinguish between categorical and numerical attributes
cat_atts = []
cont_atts = []
for col in atts:
    if df_train[col].dtype == object:
        #         df_train[col] = df_train[col].astype('category').cat.codes
        cat_atts.append(col)
    else:
        #         df_train[col] = df_train[col].astype(float)
        cont_atts.append(col)
# here we one hot encode categorical attributes
df = pd.get_dummies(df, columns=cat_atts)
# here we remove the extra spaces from the column names of the dataset
df.columns = [col.replace(" ", "") for col in df.columns]
# here we remove the "weird" characters from the column names of the dataset
df.columns = [col.replace("?", "UNK") for col in df.columns]
# here we define the final set of features to be used in the classification task
atts = [col for col in df.columns if col not in ["TARGET", "index"]]
# here we create test set using perc_test and calibration set using perc_hold
df_train_, df_test = train_test_split(
    df, random_state=42, test_size=perc_test, stratify=df["TARGET"]
)
df_train, df_cal = train_test_split(
    df_train_, random_state=42, test_size=perc_hold, stratify=df_train_["TARGET"]
)


# The selective classification framework

Selective classification allows classifiers to abstain from predicting, avoiding making mistakes over difficult instances. In this notebook we show how to use the selective classification framework to train selective classifiers and evaluate them. We use the Adult dataset, which is a binary classification task. We use the LightGBM classifier as base model for the selective classifiers. We train the selective classifiers using the training set, calibrate them using the calibration set, and evaluate them using the test set. We use the accuracy as metric for the evaluation of the selective classifiers. We also show how to use the AUC as metric for the evaluation of the selective classifiers.

More formally we have:
- $X$ are the features
- $Y$ are the target variables
- $f: X \rightarrow Y$ is the base classifier
- $g: X \rightarrow \{0,1\}$ is the selection function

The selective classifier is then the pair $(f,g)$, i.e.

\begin{equation}
(f,g)(x) = \begin{cases}
f(x) \quad \text{if } g(x) = 1 \\
\text{abstain} \quad \text{if } g(x) = 0
\end{cases}
\end{equation}

To build a selective classifier, one has to specify a target coverage $c \in [0,1]$, which is the percentage of instances that the selective classifier is allowed to abstain from predicting. The selective classifier is then calibrated to predict on $c$ fraction of the instances. Depending on the methodology, one might need a calibration set to calibrate the selective classifier to achieve the target coverage. Ideally, the more we abstain the more we reduce the number of mistakes, hence performance should improve.

In [3]:
# // Train selective classifiers
from code.model_agnostic import PlugInRule, PlugInRuleAUC, SCRoss
# here we define the base model to be used by the selective classifiers
clf_base = LGBMClassifier(random_state=42)
# here we define the selective classifiers
# plugin : plug-in rule by Herbei and Wegkamp
# plugin_auc : plug-in rule by Pugnana and Ruggieri, 2023b - a plug-in rule that improves AUC when abstaining
# scr : cross-fitting approach by Pugnana and Ruggieri, 2023a - a method that uses cross-fitting to avoid overfitting and reserve some data for calibration
plg = PlugInRule(clf_base)
plg_auc = PlugInRuleAUC(clf_base)
scr = SCRoss(clf_base)

In [4]:
# here we train the selective classifiers
plg.fit(df_train[atts], df_train["TARGET"])
plg_auc.fit(df_train[atts], df_train["TARGET"])
scr.fit(df_train_[atts], df_train_["TARGET"])

[LightGBM] [Info] Number of positive: 8180, number of negative: 26008
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002681 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 750
[LightGBM] [Info] Number of data points in the train set: 34188, number of used features: 80
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.239265 -> initscore=-1.156712
[LightGBM] [Info] Start training from score -1.156712
[LightGBM] [Info] Number of positive: 8180, number of negative: 26008
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002792 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 750
[LightGBM] [Info] Number of data points in the train set: 34188, number of used features: 80
[LightGBM] [Info] [bin

In [5]:
#calibrate selective classifiers for target coverages
target_coverages = [.99, .75, .50] # here we define the target coverages for the selective classifiers
plg.calibrate(df_cal[atts], target_coverages=target_coverages)
plg_auc.calibrate(df_cal[atts], df_cal["TARGET"], target_coverages=target_coverages)
scr.calibrate(target_coverages=target_coverages)

In [6]:
# // Evaluate selective classifiers
# here we create an array containing levels for acceptance depending on how the selective classifiers are calibrated
# since we used three target coverages, we have four levels of acceptance, i.e., 0, 1, 2, 3
selected_plg = plg.qband(df_test[atts])
selected_plg_auc = plg_auc.qband(df_test[atts])
selected_scr = scr.qband(df_test[atts])

In [7]:
# check that the selective classifiers are able to identify instances where they make less mistakes
print("PLG - full coverage: ", accuracy_score(df_test["TARGET"][selected_plg>=0], plg.predict(df_test[atts][selected_plg>=0])))
print("PLG - predicting on 99% of cases: ", accuracy_score(df_test["TARGET"][selected_plg>=1], plg.predict(df_test[atts][selected_plg>=1])))
print("PLG - predicting on 75% of cases: ", accuracy_score(df_test["TARGET"][selected_plg>=2], plg.predict(df_test[atts][selected_plg>=2])))
print("PLG - predicting on 50% of cases: ", accuracy_score(df_test["TARGET"][selected_plg>=3], plg.predict(df_test[atts][selected_plg>=3])))

PLG - full coverage:  0.8756269833145665
PLG - predicting on 99% of cases:  0.8792604833712043
PLG - predicting on 75% of cases:  0.9482135688478522
PLG - predicting on 50% of cases:  0.9895833333333334


In [8]:
print("PLG AUC - full coverage: ", accuracy_score(df_test["TARGET"][selected_plg_auc>=0], plg_auc.predict(df_test[atts][selected_plg_auc>=0])))
print("PLG AUC - predicting on 99% of cases: ", accuracy_score(df_test["TARGET"][selected_plg_auc>=1], plg_auc.predict(df_test[atts][selected_plg_auc>=1])))
print("PLG AUC - predicting on 75% of cases: ", accuracy_score(df_test["TARGET"][selected_plg_auc>=2], plg_auc.predict(df_test[atts][selected_plg_auc>=2])))
print("PLG AUC - predicting on 50% of cases: ", accuracy_score(df_test["TARGET"][selected_plg_auc>=3], plg_auc.predict(df_test[atts][selected_plg_auc>=3])))

PLG AUC - full coverage:  0.8756269833145665
PLG AUC - predicting on 99% of cases:  0.876993166287016
PLG AUC - predicting on 75% of cases:  0.916712479384277
PLG AUC - predicting on 50% of cases:  0.9800558436378142


In [9]:
print("SCR - full coverage: ", accuracy_score(df_test["TARGET"][selected_scr>=0], scr.predict(df_test[atts][selected_scr>=0])))
print("SCR - predicting on 99% of cases: ", accuracy_score(df_test["TARGET"][selected_scr>=1], scr.predict(df_test[atts][selected_scr>=1])))
print("SCR - predicting on 75% of cases: ", accuracy_score(df_test["TARGET"][selected_scr>=2], scr.predict(df_test[atts][selected_scr>=2])))
print("SCR - predicting on 50% of cases: ", accuracy_score(df_test["TARGET"][selected_scr>=3], scr.predict(df_test[atts][selected_scr>=3])))

SCR - full coverage:  0.8759340771829256
SCR - predicting on 99% of cases:  0.8794721105268585
SCR - predicting on 75% of cases:  0.9488873904248145
SCR - predicting on 50% of cases:  0.9901368760064412


In [ ]:
# check what happens when changing metric to AUC

In [33]:
print("PLG - full coverage: ", roc_auc_score(df_test["TARGET"][selected_plg>=0], plg.predict_proba(df_test[atts][selected_plg>=0])[:,1]))
print("PLG - predicting on 99% of cases: ", roc_auc_score(df_test["TARGET"][selected_plg>=1], plg.predict_proba(df_test[atts][selected_plg>=1])[:,1]))
print("PLG - predicting on 75% of cases: ", roc_auc_score(df_test["TARGET"][selected_plg>=2], plg.predict_proba(df_test[atts][selected_plg>=2])[:,1]))
print("PLG - predicting on 50% of cases: ", roc_auc_score(df_test["TARGET"][selected_plg>=3], plg.predict_proba(df_test[atts][selected_plg>=3])[:,1]))

PLG - full coverage:  0.9299814639133982
PLG - predicting on 99% of cases:  0.9314342951664598
PLG - predicting on 75% of cases:  0.9598424240705253
PLG - predicting on 50% of cases:  0.9884235349349638


In [32]:
print("PLG AUC - full coverage: ", roc_auc_score(df_test["TARGET"][selected_plg_auc>=0], plg_auc.predict_proba(df_test[atts][selected_plg_auc>=0])[:,1]))
print("PLG AUC - predicting on 99% of cases: ", roc_auc_score(df_test["TARGET"][selected_plg_auc>=1], plg_auc.predict_proba(df_test[atts][selected_plg_auc>=1])[:,1]))
print("PLG AUC - predicting on 75% of cases: ", roc_auc_score(df_test["TARGET"][selected_plg_auc>=2], plg_auc.predict_proba(df_test[atts][selected_plg_auc>=2])[:,1]))
print("PLG AUC - predicting on 50% of cases: ", roc_auc_score(df_test["TARGET"][selected_plg_auc>=3], plg_auc.predict_proba(df_test[atts][selected_plg_auc>=3])[:,1]))

PLG AUC - full coverage:  0.9299814639133982
PLG AUC - predicting on 99% of cases:  0.9319424790167712
PLG AUC - predicting on 75% of cases:  0.9684671594950266
PLG AUC - predicting on 50% of cases:  0.9919272689795878
